In [1]:
import cv2
import numpy as np

frame_num = 177
frame2 = cv2.imread('./dataset1T/dataset_1_0.jpg')

def set_frame_number(x):
    global frame_num
    frame_num = x
    return

result = np.zeros((int(frame2.shape[0])//4,int(frame2.shape[1])//4*5,3))
count  = np.zeros((int(frame2.shape[0])//4,int(frame2.shape[1])//4*5))
ones = np.ones(((int(frame2.shape[0])//4,int(frame2.shape[1])//4)))

cv2.namedWindow('matching')
cv2.createTrackbar('frame no.','matching',0,frame_num-1,set_frame_number)

kpdetector = cv2.xfeatures2d.SIFT_create()
#kpdetector = cv2.AKAZE_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
while frame_num > -1:
    cv2.setTrackbarPos('frame no.','matching',frame_num)
    
    frame2 = cv2.imread('./dataset1T/dataset_1_%d.jpg' % frame_num)
    frame2 = cv2.resize(frame2,(frame2.shape[1]//4,frame2.shape[0]//4))
    
    #kp1, dt1 = kpdetector.detectAndCompute(frame1,None)
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    kp2 = kpdetector.detect(gray,None)
    dt2 = kpdetector.compute(gray,kp2)[1]
    if frame_num == 176:
        T = np.eye(3)
#         T[0,2] = 0
#         T[1,2] = 0
        result = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count= cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        count += t_count.astype(np.float)
        disp = result.copy()
        cv2.imshow('stitched image',disp.astype(np.uint8))
    
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2
    else:
        # Match descriptors.
        matches = bf.match(dt2,dt1)
        print('{}, # of matches:{}'.format(frame_num,len(matches)))

        # Sort in ascending order of distance.
        matches = sorted(matches, key = lambda x:x.distance)
        
        src = []
        dst = []
        for m in matches:
            src.append(kp2[m.queryIdx].pt + (1,))
            dst.append(kp1[m.trainIdx].pt + (1,))
            
        src = np.array(src,dtype=np.float)
        dst = np.array(dst,dtype=np.float)
    
        # find a homography to map src to dst
        A, mask = cv2.findHomography(src, dst, cv2.RANSAC)
        
        # map to the first frame
        T = T.dot(A)
        warp_img = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count  = cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        result+= warp_img
        count += t_count.astype(np.float)

        t_count= count.copy()
        t_count[t_count == 0] = 1
        disp = result.copy()
        
        disp[:,:,0] = result[:,:,0] / t_count
        disp[:,:,1] = result[:,:,1] / t_count
        disp[:,:,2] = result[:,:,2] / t_count
 
        cv2.imshow('stitched image',disp.astype(np.uint8))
   
        cv2.imshow('matching',cv2.drawMatches(frame2,kp2,frame1,kp1,matches[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        
        
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2

    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    frame_num -= 1   
cv2.destroyAllWindows()

175, # of matches:388
174, # of matches:377
173, # of matches:358
172, # of matches:384
171, # of matches:351
170, # of matches:368
169, # of matches:408
168, # of matches:371
167, # of matches:367
166, # of matches:365
165, # of matches:352
164, # of matches:348
163, # of matches:347
162, # of matches:367
161, # of matches:374
160, # of matches:351
159, # of matches:333
158, # of matches:353
157, # of matches:369
156, # of matches:389


In [ ]:
import cv2
import numpy as np

cap    = cv2.VideoCapture('hw4_dataset1.mp4')
print('height:{} width:{}'.format(int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))))
frame_num = 0
total_frame=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

kpdetector = cv2.xfeatures2d.SIFT_create() 
#kpdetector = cv2.AKAZE_create()

while frame_num < total_frame:
    ret, img = cap.read()
    cv2.imwrite('./dataset1T/dataset_1_'+str(frame_num)+'.jpg', img)
    frame_num+=1
cap.release()
cv2.destroyAllWindows()

In [14]:
import cv2
import numpy as np

totalFrame = 177
frame2 = cv2.imread('./dataset1T/dataset_1_0.jpg') #毒第一張圖片以便知道圖片大小
forwardFrameNum = totalFrame // 2 - 25 #向左第一章圖片的位置(中間偏右)
backwardFrameNum = totalFrame // 2 - 25 #向右第一章圖片的位置(中間偏右)

def set_frame_number(x):
    global frame_num
    frame_num = x
    return

def matchesAndHomography(dt1, dt2, kp1, kp2, frameNum):  #計算向左或向右的matches
    # Match descriptors.
    matches = bf.match(dt2,dt1)

    print('FrameNum: {}, # of matches:{}'.format(frameNum,len(matches)))


    # Sort in ascending order of distance.
    matches = sorted(matches, key = lambda x:x.distance)

    src = []
    dst = []
    for m in matches:
        src.append(kp2[m.queryIdx].pt + (1,))
        dst.append(kp1[m.trainIdx].pt + (1,))

    src = np.array(src,dtype=np.float)
    dst = np.array(dst,dtype=np.float)
    # find a homography to map src to dst
    A, mask = cv2.findHomography(src, dst, cv2.RANSAC)
    return A, matches

result = np.zeros((int(frame2.shape[0])//4,int(frame2.shape[1])//4*3,3))
count  = np.zeros((int(frame2.shape[0])//4,int(frame2.shape[1])//4*3))
ones = np.ones(((int(frame2.shape[0])//4,int(frame2.shape[1])//4)))

cv2.namedWindow('matchingForward') #向左的matches圖
cv2.createTrackbar('frame no.','matchingForward',forwardFrameNum,totalFrame-1,set_frame_number)
cv2.namedWindow('matchingBackward')#向右的matches圖
cv2.createTrackbar('frame no.','matchingBackward',backwardFrameNum,totalFrame-1,set_frame_number)

kpdetector = cv2.xfeatures2d.SIFT_create()
#kpdetector = cv2.AKAZE_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
while forwardFrameNum < totalFrame and backwardFrameNum > -1:
    cv2.setTrackbarPos('frame no.','matchingForward',forwardFrameNum)
    cv2.setTrackbarPos('frame no.','matchingBackward',backwardFrameNum)
    frame2Forward = cv2.imread('./dataset1T/dataset_1_%d.jpg' % forwardFrameNum) #向左讀取照片
    frame2Forward = cv2.resize(frame2Forward,(frame2Forward.shape[1]//4,frame2Forward.shape[0]//4))
    frame2Backward = cv2.imread('./dataset1T/dataset_1_%d.jpg' % backwardFrameNum)#向右讀取照片
    frame2Backward = cv2.resize(frame2Backward,(frame2Backward.shape[1]//4,frame2Backward.shape[0]//4))
    
    
    gray = cv2.cvtColor(frame2Forward, cv2.COLOR_BGR2GRAY)
    kp2Forward = kpdetector.detect(gray,None)
    dt2Forward = kpdetector.compute(gray,kp2Forward)[1] #對向右一張的圖片產生key point和detection
    
    gray = cv2.cvtColor(frame2Backward, cv2.COLOR_BGR2GRAY)
    kp2Backward = kpdetector.detect(gray,None)
    dt2Backward = kpdetector.compute(gray,kp2Backward)[1]#對向右=左一張的圖片產生key point和detection
    
    if forwardFrameNum == totalFrame // 2 - 25: #第一張圖片直接擺上結果
        TForward = np.eye(3)
        TForward[0,2] = result.shape[1] / 2 - frame2Forward.shape[1] * 0.6 #第一張圖片擺的橫向位置
        TForward[1,2] = 0 #第一張圖片擺縱橫向位置
        
        TBackward = np.eye(3)
        TBackward[0,2] = result.shape[1] / 2 - frame2Forward.shape[1] * 0.6
        TBackward[1,2] = 0
        
        
        result = cv2.warpPerspective(frame2Forward,TForward,(result.shape[1],result.shape[0])).astype(np.float)
        tCount= cv2.warpPerspective(ones,TForward,(result.shape[1],result.shape[0])).astype(np.float)
        count += tCount.astype(np.float)
        disp = result.copy()
        cv2.imshow('stitched image',disp.astype(np.uint8))
        
    
        frame1Forward = frame2Forward
        kp1Forward = kp2Forward
        dt1Forward = dt2Forward
        
        frame1Backward = frame2Backward
        kp1Backward = kp2Backward
        dt1Backward = dt2Backward
    else:
        AForward, matchesForward = matchesAndHomography(dt1Forward, dt2Forward, kp1Forward, kp2Forward, forwardFrameNum)
        # 向右找出A和matches
        TForward = TForward.dot(AForward)
        warp_img = cv2.warpPerspective(frame2Forward,TForward,(result.shape[1],result.shape[0])).astype(np.float)
        tCount  = cv2.warpPerspective(ones,TForward,(result.shape[1],result.shape[0])).astype(np.float)
        result+= warp_img
        count += tCount.astype(np.float)
        
        ABackward, matchesBackward = matchesAndHomography(dt1Backward, dt2Backward, kp1Backward, kp2Backward, backwardFrameNum)
        # 向左找出A和matches
        TBackward = TBackward.dot(ABackward)
        warp_img = cv2.warpPerspective(frame2Backward,TBackward,(result.shape[1],result.shape[0])).astype(np.float)
        tCount  = cv2.warpPerspective(ones,TBackward,(result.shape[1],result.shape[0])).astype(np.float)
        result+= warp_img
        count += tCount.astype(np.float)

        tCount= count.copy()
        tCount[tCount == 0] = 1
        disp = result.copy()
        
        disp[:,:,0] = result[:,:,0] / tCount
        disp[:,:,1] = result[:,:,1] / tCount
        disp[:,:,2] = result[:,:,2] / tCount
 
        cv2.imshow('stitched image',disp.astype(np.uint8))
   
        cv2.imshow('matchingForward',cv2.drawMatches(frame2Forward,kp2Forward,frame1Forward,kp1Forward,matchesForward[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        #上一張圖(右邊)與下一張圖(左邊)的key point 比對圖
        cv2.imshow('matchingBackward',cv2.drawMatches(frame2Backward,kp2Backward,frame1Backward,kp1Backward,matchesBackward[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        #上一張圖(左邊)與下一張圖(右邊)的key point 比對圖
        
        
        frame1Forward = frame2Forward
        kp1Forward = kp2Forward
        dt1Forward = dt2Forward
        
        frame1Backward = frame2Backward
        kp1Backward = kp2Backward
        dt1Backward = dt2Backward

    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    forwardFrameNum += 1
    backwardFrameNum -= 1
cv2.imwrite("result.jpg", disp.astype(np.uint8))
cv2.waitKey()    
cv2.destroyAllWindows()

FrameNum: 64, # of matches:418
FrameNum: 62, # of matches:449
FrameNum: 65, # of matches:428
FrameNum: 61, # of matches:442
FrameNum: 66, # of matches:414
FrameNum: 60, # of matches:448
FrameNum: 67, # of matches:402
FrameNum: 59, # of matches:433
FrameNum: 68, # of matches:431
FrameNum: 58, # of matches:432
FrameNum: 69, # of matches:454
FrameNum: 57, # of matches:431
FrameNum: 70, # of matches:465
FrameNum: 56, # of matches:421
FrameNum: 71, # of matches:447
FrameNum: 55, # of matches:450
FrameNum: 72, # of matches:434
FrameNum: 54, # of matches:415
FrameNum: 73, # of matches:424
FrameNum: 53, # of matches:419
FrameNum: 74, # of matches:441
FrameNum: 52, # of matches:409
FrameNum: 75, # of matches:436
FrameNum: 51, # of matches:378
FrameNum: 76, # of matches:461
FrameNum: 50, # of matches:406
FrameNum: 77, # of matches:453
FrameNum: 49, # of matches:420
FrameNum: 78, # of matches:435
FrameNum: 48, # of matches:446
FrameNum: 79, # of matches:430
FrameNum: 47, # of matches:422
FrameNum

In [2]:
import cv2
import numpy as np

totalFrame = 177
frame2 = cv2.imread('./dataset1T/dataset_1_0.jpg') #讀第一張圖片以便知道圖片大小
forwardFrameNum = totalFrame // 2 - 25 #向左第一章圖片的位置(中間偏右)
backwardFrameNum = totalFrame // 2 - 25 #向右第一章圖片的位置(中間偏右)

def set_frame_number(x):
    global frame_num
    frame_num = x
    return

def matchesAndHomography(dt1, dt2, kp1, kp2, frameNum):  #計算向左或向右的matches
    # Match descriptors.
    matches = bf.match(dt2,dt1)

    print('FrameNum: {}, # of matches:{}'.format(frameNum,len(matches)))


    # Sort in ascending order of distance.
    matches = sorted(matches, key = lambda x:x.distance)

    src = []
    dst = []
    for m in matches:
        src.append(kp2[m.queryIdx].pt + (1,))
        dst.append(kp1[m.trainIdx].pt + (1,))

    src = np.array(src,dtype=np.float)
    dst = np.array(dst,dtype=np.float)
    # find a homography to map src to dst
    A, mask = cv2.findHomography(src, dst, cv2.RANSAC)
    return A, matches

result = np.zeros((int(frame2.shape[0])//4,int(frame2.shape[1])//4*3,3))
count  = np.zeros((int(frame2.shape[0])//4,int(frame2.shape[1])//4*3))
ones = np.ones(((int(frame2.shape[0])//4,int(frame2.shape[1])//4)))

#cv2.namedWindow('matchingForward') #向左的matches圖
cv2.createTrackbar('frame no.','matchingForward',forwardFrameNum,totalFrame-1,set_frame_number)
#cv2.namedWindow('matchingBackward')#向右的matches圖
cv2.createTrackbar('frame no.','matchingBackward',backwardFrameNum,totalFrame-1,set_frame_number)

kpdetector = cv2.xfeatures2d.SIFT_create()
#kpdetector = cv2.AKAZE_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
while forwardFrameNum < totalFrame or backwardFrameNum > -1:
    if forwardFrameNum < totalFrame:
        cv2.setTrackbarPos('frame no.','matchingForward',forwardFrameNum)
        frame2Forward = cv2.imread('./dataset1T/dataset_1_%d.jpg' % forwardFrameNum) #向左讀取照片
        frame2Forward = cv2.resize(frame2Forward,(frame2Forward.shape[1]//4,frame2Forward.shape[0]//4))
        gray = cv2.cvtColor(frame2Forward, cv2.COLOR_BGR2GRAY)
        kp2Forward = kpdetector.detect(gray,None)
        dt2Forward = kpdetector.compute(gray,kp2Forward)[1] #對向右一張的圖片產生key point和detection
    if backwardFrameNum > -1:
        cv2.setTrackbarPos('frame no.','matchingBackward',backwardFrameNum)
        frame2Backward = cv2.imread('./dataset1T/dataset_1_%d.jpg' % backwardFrameNum)#向右讀取照片
        frame2Backward = cv2.resize(frame2Backward,(frame2Backward.shape[1]//4,frame2Backward.shape[0]//4))
        gray = cv2.cvtColor(frame2Backward, cv2.COLOR_BGR2GRAY)
        kp2Backward = kpdetector.detect(gray,None)
        dt2Backward = kpdetector.compute(gray,kp2Backward)[1]#對向右=左一張的圖片產生key point和detection      
    
   
    
    if forwardFrameNum == totalFrame // 2 - 25: #第一張圖片直接擺上結果
        TForward = np.eye(3)
        TForward[0,2] = result.shape[1] / 2 - frame2Forward.shape[1] * 0.2 #第一張圖片擺的橫向位置
        TForward[1,2] = 0 #第一張圖片擺縱橫向位置
        
        TBackward = np.eye(3)
        TBackward[0,2] = result.shape[1] / 2 - frame2Forward.shape[1] * 0.2
        TBackward[1,2] = 0
        
        
        result = cv2.warpPerspective(frame2Forward,TForward,(result.shape[1],result.shape[0])).astype(np.float)
        tCount= cv2.warpPerspective(ones,TForward,(result.shape[1],result.shape[0])).astype(np.float)
        count += tCount.astype(np.float)
        disp = result.copy()
        cv2.imshow('stitched image',disp.astype(np.uint8))
        
    
        frame1Forward = frame2Forward
        kp1Forward = kp2Forward
        dt1Forward = dt2Forward
        
        frame1Backward = frame2Backward
        kp1Backward = kp2Backward
        dt1Backward = dt2Backward
    else:
        if forwardFrameNum < totalFrame:
            AForward, matchesForward = matchesAndHomography(dt1Forward, dt2Forward, kp1Forward, kp2Forward, forwardFrameNum)
            # 向右找出A和matches
            TForward = TForward.dot(AForward)
            warp_img = cv2.warpPerspective(frame2Forward,TForward,(result.shape[1],result.shape[0])).astype(np.float)
            tCount  = cv2.warpPerspective(ones,TForward,(result.shape[1],result.shape[0])).astype(np.float)
            result+= warp_img
            count += tCount.astype(np.float)
            
        if backwardFrameNum > -1:
            ABackward, matchesBackward = matchesAndHomography(dt1Backward, dt2Backward, kp1Backward, kp2Backward, backwardFrameNum)
            # 向左找出A和matches
            TBackward = TBackward.dot(ABackward)
            warp_img = cv2.warpPerspective(frame2Backward,TBackward,(result.shape[1],result.shape[0])).astype(np.float)
            tCount  = cv2.warpPerspective(ones,TBackward,(result.shape[1],result.shape[0])).astype(np.float)
            result+= warp_img
            count += tCount.astype(np.float)

        tCount= count.copy()
        tCount[tCount == 0] = 1
        disp = result.copy()
        
        disp[:,:,0] = result[:,:,0] / tCount
        disp[:,:,1] = result[:,:,1] / tCount
        disp[:,:,2] = result[:,:,2] / tCount
 
        cv2.imshow('stitched image',disp.astype(np.uint8))
   
        cv2.imshow('matchingForward',cv2.drawMatches(frame2Forward,kp2Forward,frame1Forward,kp1Forward,matchesForward[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        #上一張圖(右邊)與下一張圖(左邊)的key point 比對圖
        cv2.imshow('matchingBackward',cv2.drawMatches(frame2Backward,kp2Backward,frame1Backward,kp1Backward,matchesBackward[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        #上一張圖(左邊)與下一張圖(右邊)的key point 比對圖
        
        
        frame1Forward = frame2Forward
        kp1Forward = kp2Forward
        dt1Forward = dt2Forward
        
        frame1Backward = frame2Backward
        kp1Backward = kp2Backward
        dt1Backward = dt2Backward

    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    forwardFrameNum += 1
    backwardFrameNum -= 1
cv2.imwrite("result.jpg", disp.astype(np.uint8))
cv2.waitKey()    
cv2.destroyAllWindows()

FrameNum: 64, # of matches:418
FrameNum: 62, # of matches:449
FrameNum: 65, # of matches:428
FrameNum: 61, # of matches:442
FrameNum: 66, # of matches:414
FrameNum: 60, # of matches:448
FrameNum: 67, # of matches:402
FrameNum: 59, # of matches:433
FrameNum: 68, # of matches:431
FrameNum: 58, # of matches:432
FrameNum: 69, # of matches:454
FrameNum: 57, # of matches:431
FrameNum: 70, # of matches:465
FrameNum: 56, # of matches:421
FrameNum: 71, # of matches:447
FrameNum: 55, # of matches:450
FrameNum: 72, # of matches:434
FrameNum: 54, # of matches:415
FrameNum: 73, # of matches:424
FrameNum: 53, # of matches:419
FrameNum: 74, # of matches:441
FrameNum: 52, # of matches:409
FrameNum: 75, # of matches:436
FrameNum: 51, # of matches:378
FrameNum: 76, # of matches:461
FrameNum: 50, # of matches:406
FrameNum: 77, # of matches:453
FrameNum: 49, # of matches:420
FrameNum: 78, # of matches:435
FrameNum: 48, # of matches:446
FrameNum: 79, # of matches:430
FrameNum: 47, # of matches:422
FrameNum